In [21]:
import pandas as pd
import numpy as np
import re

In [2]:
# IMPORTAÇÃO DA BASE DE SÓCIOS
socios_df = pd.read_csv('C:\\DATASETS\\CNPJ\\2-processado\\socios.csv', sep=',', dtype=str)

In [3]:
# Conferindo campos e tipos
socios_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27729558 entries, 0 to 27729557
Data columns (total 12 columns):
cnpj                 object
tipo_socio           object
nome_socio           object
cnpj_cpf_socio       object
cod_qualificacao     object
perc_capital         object
data_entrada         object
cod_pais_ext         object
nome_pais_ext        object
cpf_repres           object
nome_repres          object
cod_qualif_repres    object
dtypes: object(12)
memory usage: 2.5+ GB


In [4]:
# Conferindo campos e valores
socios_df.describe

<bound method NDFrame.describe of                     cnpj tipo_socio                        nome_socio  \
0         00000000000191          2          MARCIO HAMILTON FERREIRA   
1         00000000000191          2         NILSON MARTINIANO MOREIRA   
2         00000000000191          2             WALTER MALIENI JUNIOR   
3         00000000000191          2       CARLOS ALBERTO ARAUJO NETTO   
4         00000000000191          2          ANTONIO MAURICIO MAURANO   
...                  ...        ...                               ...   
27729553  98819402000117          2  ANGELA VIRGINIA CORTONI MICHELON   
27729554  98819402000117          2                  ANTONIO MICHELON   
27729555  98819402000206          2                  ANTONIO MICHELON   
27729556  98819402000206          2  ANGELA VIRGINIA CORTONI MICHELON   
27729557  99017782000139          2   FRANCISCO FREIRE DE VASCONCELOS   

         cnpj_cpf_socio cod_qualificacao perc_capital data_entrada  \
0           ***9236

Identificando percentual de valores ausentes por coluna:

In [5]:
(socios_df.isnull().sum() / socios_df.shape[0] * 100)
   .sort_values(ascending=False)

nome_pais_ext        100.000000
cod_pais_ext         100.000000
nome_repres           99.141003
cpf_repres            96.110032
cod_qualif_repres      0.000000
data_entrada           0.000000
perc_capital           0.000000
cod_qualificacao       0.000000
cnpj_cpf_socio         0.000000
nome_socio             0.000000
tipo_socio             0.000000
cnpj                   0.000000
dtype: float64

Conferindo se o cnpj_cpf_socio contém realmente apenas CPF(11 dígitos) ou CNPJ(14 dígitos). Também fornece informação sobre quantas pessoas físicas e quantas pessoas jurídicas são sócios:

In [6]:
socios_df.cnpj_cpf_socio.str.len().value_counts()

11    26908065
14      821493
Name: cnpj_cpf_socio, dtype: int64

Os valores estão conforme esperado: 11 dígitos para CPF e 14 dígitos para CNPJ. Essa é a forma clássica trabalhada internamente na Receita Federal.

In [25]:
# IMPORTAÇÃO DA BASE DE EMPRESAS ROTULADAS
empresas_rotuladas_df = pd.read_csv(r'C:\DATASETS\Empresas Rotuladas\empresas_rotuladas.csv',
                                    sep=',', dtype=str)
empresas_rotuladas_df

,cnpj,matriz_filial,razao_social,nome_fantasia,situacao,data_situacao,motivo_situacao,cod_nat_juridica,data_inicio_ativ,cnae_fiscal,...,telefone_1,email,qualif_resp,capital_social,porte,opc_simples,opc_mei,inexistente,anos_de_atividade_empresa,telefone_provavelmente_invalido
0,00000942000170,1,NILZA LAVISO ARAUJO,MIL CARIMBOS,08,20180201,67,2135,19940505,3299002,...,NaN,NaN,50,0.0,01,6,False,True,24,False
1,00001102000121,1,JOSE GERALDO PEREIRA SANTANA ME,NaN,08,20180201,67,2135,19940505,4729699,...,NaN,NaN,50,0.0,01,6,False,True,24,False
2,00005201000181,1,PETRO-ALFA COMERCIO DE DERIVADOS DE PETROLEO LTDA,NaN,08,20180326,15,2062,19940517,4681802,...,81651700,petroalfa@petroalfa.cnt.br,49,0.0,01,0,False,True,24,False
3,00005201000262,2,PETRO-ALFA COMERCIO DE DERIVADOS DE PETROLEO LTDA,PETRO-ALFA COMERCIO DE DERIVADOS DE PETROLEO,08,20180326,15,2062,20011211,4681802,...,33380127,PETROALFA@PETROALFA.COM.BR,49,0.0,01,0,False,True,17,False
4,00010501000159,1,B S PETRO COM E DIST DE DERIVADOS DE PETROLEO ...,B S PETRO,08,20180326,15,2062,19940527,4681801,...,NaN,NaN,49,0.0,01,0,False,True,24,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1559938,98593668000194,1,AUTO VIACAO VENANCIO AIRES LTDA,VIASUL,02,20040515,00,2062,19680327,4922102,...,37931700,VIASUL@VIASULBUS.COM.BR,49,2000000.0,05,0,False,False,53,False
1559939,98669997000252,2,E. R. AMANTINO INDUSTRIA METALURGICA LTDA,MICROVERA,02,20051103,00,2062,19820303,2451200,...,34419100,CONTABILIDADE@ERAMANTINO.COM.BR,49,0.0,05,0,False,False,39,False
1559940,98670235000195,1,MAZZAROLLO - AGRO MAQUINAS LTDA,NaN,02,20020720,00,2062,19660824,4789099,...,NaN,NaN,49,1600000.0,05,0,False,False,55,False
1559941,98670656000116,1,RESTAURANTE FARINA LTDA,NaN,02,20050416,00,2062,19660823,5611201,...,NaN,NaN,49,0.0,01,5,False,False,55,False


In [26]:
# Separa os sócios das empresas 
socios_empresas_rotuladas_df = socios_df[socios_df.cnpj.isin(empresas_rotuladas_df.cnpj)]
socios_empresas_rotuladas_df

,cnpj,tipo_socio,nome_socio,cnpj_cpf_socio,cod_qualificacao,perc_capital,data_entrada,cod_pais_ext,nome_pais_ext,cpf_repres,nome_repres,cod_qualif_repres
0,00000000000191,2,MARCIO HAMILTON FERREIRA,***923641**,10,0.0,20101117,NaN,NaN,NaN,NaN,00
1,00000000000191,2,NILSON MARTINIANO MOREIRA,***491386**,10,0.0,20101117,NaN,NaN,NaN,NaN,00
2,00000000000191,2,WALTER MALIENI JUNIOR,***718468**,10,0.0,20101117,NaN,NaN,NaN,NaN,00
3,00000000000191,2,CARLOS ALBERTO ARAUJO NETTO,***415907**,10,0.0,20120327,NaN,NaN,NaN,NaN,00
4,00000000000191,2,ANTONIO MAURICIO MAURANO,***022878**,10,0.0,20120327,NaN,NaN,NaN,NaN,00
...,...,...,...,...,...,...,...,...,...,...,...,...
27729264,98670235000195,2,UGO MAZZAROLO,***835910**,49,0.0,19880127,NaN,NaN,NaN,NaN,00
27729265,98670235000195,1,MAZZA DO BRASIL - COMERCIO E PARTICIPACOES LTDA,08443821000104,22,0.0,20090616,NaN,NaN,20772939004,NaN,05
27729274,98670656000116,2,RAFAEL FARINA,***260760**,49,0.0,20000201,NaN,NaN,NaN,NaN,00
27729275,98670656000116,2,ALCIANE MARIA RIGO FARINA,***035160**,49,0.0,20140731,NaN,NaN,NaN,NaN,00


In [27]:
cnpj_sem_socios = set(empresas_rotuladas_df.cnpj).difference(socios_empresas_rotuladas_df.cnpj)

In [28]:
len(cnpj_sem_socios)

1433570

Temos 1.433.570 empresas sem sócios descritos na base nacional. Vamos tentar recuperar parte deles:

In [29]:
empresas_sem_socio_df =
   empresas_rotuladas_df[empresas_rotuladas_df.cnpj.isin(cnpj_sem_socios)]
empresas_sem_socio_df

,cnpj,matriz_filial,razao_social,nome_fantasia,situacao,data_situacao,motivo_situacao,cod_nat_juridica,data_inicio_ativ,cnae_fiscal,...,telefone_1,email,qualif_resp,capital_social,porte,opc_simples,opc_mei,inexistente,anos_de_atividade_empresa,telefone_provavelmente_invalido
0,00000942000170,1,NILZA LAVISO ARAUJO,MIL CARIMBOS,08,20180201,67,2135,19940505,3299002,...,NaN,NaN,50,0.0,01,6,False,True,24,False
1,00001102000121,1,JOSE GERALDO PEREIRA SANTANA ME,NaN,08,20180201,67,2135,19940505,4729699,...,NaN,NaN,50,0.0,01,6,False,True,24,False
4,00010501000159,1,B S PETRO COM E DIST DE DERIVADOS DE PETROLEO ...,B S PETRO,08,20180326,15,2062,19940527,4681801,...,NaN,NaN,49,0.0,01,0,False,True,24,False
5,00014369000153,1,LEONIDES PEREIRA,BAR SAO PAULO APOSTOLO,08,20180201,67,2135,19940609,4712100,...,36350123,agendaserv@terra.com.br,50,0.0,01,6,False,True,24,False
6,00022120000190,1,LUIZ CARLOS DE CARVALHO INOJOSA 10353093866,INOSERVICE ELETRONICA,08,20180201,67,2135,19940406,4757100,...,96327398,edilson.lara1@itelefonica.com.br,50,0.0,01,6,False,True,24,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1559848,97553710000180,1,VICTOR LUIZ SOUSA DA SILVA,CUIABATUR,02,20110714,00,2135,20110714,7911200,...,30543888,vitor_ldf@hotmail.com,50,1.0,01,5,False,False,10,False
1559850,97554278000142,1,JACQUELINE TERCILIA LEANDRO - MOTO PECAS,G MOTOS,02,20110615,00,2135,20110615,4541206,...,32277977,excel@octexcel.com.br,50,10000.0,01,5,False,False,10,False
1559851,97554394000161,1,"DANIEL PEREIRA PROMOCAO DE SHOWS, BANDAS E EVE...",DAN PEREIRA EVENTOS,02,20110714,00,2135,20110714,8230001,...,66966686,danielbspg@gmail.com,50,1000.0,01,5,True,False,10,False
1559853,97554547000170,1,T. R. HEIN TELECOM,PSIU TELECOM,02,20110714,00,2135,20110714,6110803,...,32364542,THOMAS@GRUPONSTELECOM.COM.BR,50,50000.0,01,5,False,False,10,False


In [46]:
lista_socios = list()

for index, row in empresas_sem_socio_df.iterrows():
    m = re.search('(\d{11})', row["razao_social"])
    if m:
        lista_socios.append({'cnpj': row["cnpj"], 'nome_socio': row["razao_social"].replace(m.group(0), '')
                    .strip(), 'cnpj_cpf_socio': m.group(0), 'data_entrada': row["data_inicio_ativ"]})

lista_socios_df = pd.DataFrame(lista_socios, columns=['cnpj','nome_socio','cnpj_cpf_socio','data_entrada'])
lista_socios_df

,cnpj,nome_socio,cnpj_cpf_socio,data_entrada
0,00022120000190,LUIZ CARLOS DE CARVALHO INOJOSA,10353093866,19940406
1,00070495000125,FRANCISCA GOMES DE ALBUQUERQUE,16790448349,19940519
2,00106863000148,GERALDO JOAO DOS SANTOS,15027708904,19940701
3,00128809000101,JOSE NEVES DIAS,04753540871,19940715
4,00143801000106,GASPAR AFONSO DOS SANTOS SERKES,57460795968,19940812
...,...,...,...,...
1381539,97543869000114,MARIO JORGE LIMA DA SILVA,72399317491,20110713
1381540,97548003000104,CARLOS ADRIANO ROSA DA SILVA,03028898330,20110713
1381541,97548058000106,ZIZANEIDE DE JESUS SANTOS,83199462553,20110713
1381542,97548091000136,ANTONIO CARLOS TEIXEIRA,14762284858,20110714


In [48]:
lista_socios_df.shape[0]*100/empresas_sem_socio_df.shape[0]

96.32525454087819

Conseguimos recuperar 96% dos sócios para as empresas que estão sem sócio na base nacional. Vamos agora consolidar o resultado:

In [50]:
# Primeiro, vamos ver se há alguma interseção, para evitarmos sobrescrita:
len(set(socios_empresas_rotuladas_df.cnpj).intersection(lista_socios_df.cnpj))

0

In [59]:
# Podemos então contatenar os dataframes:
print(socios_empresas_rotuladas_df.shape)
print(lista_socios_df.shape)
socios_empresas_rotuladas_df = socios_empresas_rotuladas_df.append(lista_socios_df, ignore_index=True)
print(socios_empresas_rotuladas_df.shape)

(302145, 12)
(1381544, 4)
(1683689, 12)


In [60]:
socios_empresas_rotuladas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1683689 entries, 0 to 1683688
Data columns (total 12 columns):
cnpj                 1683689 non-null object
cnpj_cpf_socio       1683689 non-null object
cod_pais_ext         0 non-null object
cod_qualif_repres    302145 non-null object
cod_qualificacao     302145 non-null object
cpf_repres           21304 non-null object
data_entrada         1683689 non-null object
nome_pais_ext        0 non-null object
nome_repres          2710 non-null object
nome_socio           1683689 non-null object
perc_capital         302145 non-null object
tipo_socio           302145 non-null object
dtypes: object(12)
memory usage: 154.1+ MB


In [61]:
len(set(empresas_rotuladas_df.cnpj).difference(set(socios_empresas_rotuladas_df.cnpj)))

52647

In [62]:
52647 * 100 / len(set(empresas_rotuladas_df.cnpj))

3.376420149366204

Ficamos com 3,37% de empresesas rotuladas sem sócios identificados. Aceitável. Prossigamos.

In [63]:
# Salvando resultado:
socios_empresas_rotuladas_df.to_csv(r'C:\DATASETS\Empresas Rotuladas\socios_empresas_rotuladas.csv', index = False)

Temos agora as duas bases principais: empresas_rotuladas.csv e socios_empresas_rotuladas.csv

In [64]:
socios_empresas_rotuladas_df.describe

<bound method NDFrame.describe of                    cnpj cnpj_cpf_socio cod_pais_ext cod_qualif_repres  \
0        00000000000191    ***923641**          NaN                00   
1        00000000000191    ***491386**          NaN                00   
2        00000000000191    ***718468**          NaN                00   
3        00000000000191    ***415907**          NaN                00   
4        00000000000191    ***022878**          NaN                00   
...                 ...            ...          ...               ...   
1683684  97543869000114    72399317491          NaN               NaN   
1683685  97548003000104    03028898330          NaN               NaN   
1683686  97548058000106    83199462553          NaN               NaN   
1683687  97548091000136    14762284858          NaN               NaN   
1683688  97549681000183    90619544449          NaN               NaN   

        cod_qualificacao cpf_repres data_entrada nome_pais_ext nome_repres  \
0          

In [65]:
socios_empresas_rotuladas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1683689 entries, 0 to 1683688
Data columns (total 12 columns):
cnpj                 1683689 non-null object
cnpj_cpf_socio       1683689 non-null object
cod_pais_ext         0 non-null object
cod_qualif_repres    302145 non-null object
cod_qualificacao     302145 non-null object
cpf_repres           21304 non-null object
data_entrada         1683689 non-null object
nome_pais_ext        0 non-null object
nome_repres          2710 non-null object
nome_socio           1683689 non-null object
perc_capital         302145 non-null object
tipo_socio           302145 non-null object
dtypes: object(12)
memory usage: 154.1+ MB


In [66]:
(socios_empresas_rotuladas_df.isnull().sum() / socios_empresas_rotuladas_df.shape[0] * 100)
   .sort_values(ascending=False)

nome_pais_ext        100.000000
cod_pais_ext         100.000000
nome_repres           99.839044
cpf_repres            98.734683
tipo_socio            82.054584
perc_capital          82.054584
cod_qualificacao      82.054584
cod_qualif_repres     82.054584
nome_socio             0.000000
data_entrada           0.000000
cnpj_cpf_socio         0.000000
cnpj                   0.000000
dtype: float64

In [67]:
# Remover colunas com alto índice de nulos (>50%):
socios_empresas_rotuladas_df.drop(columns=['nome_pais_ext', 'cod_pais_ext', 'nome_repres', 'cpf_repres', 'tipo_socio', 'perc_capital', 'cod_qualificacao', 'cod_qualif_repres'], inplace=True)

### Feature Engineering

### Transormar data_entrada em tempo_de_sociedade:

In [78]:
socios_empresas_rotuladas_df.shape

(1683689, 4)

In [127]:
socios_empresas_rotuladas_df = socios_empresas_rotuladas_df.merge(empresas_rotuladas_df[['cnpj', 'situacao', 'data_situacao']], on='cnpj', how='left')

In [129]:
socios_empresas_rotuladas_df["tempo_de_sociedade"] = round((np.where(socios_empresas_rotuladas_df.situacao=='08', socios_empresas_rotuladas_df.data_situacao.astype(int), 20210401) - socios_empresas_rotuladas_df.data_entrada.astype(int) ) / 10000).astype(int)
socios_empresas_rotuladas_df["tempo_de_sociedade"] = np.where(socios_empresas_rotuladas_df["tempo_de_sociedade"]<0,0,socios_empresas_rotuladas_df["tempo_de_sociedade"])
socios_empresas_rotuladas_df.drop(columns=['data_entrada', 'situacao', 'data_situacao'], inplace=True)
socios_empresas_rotuladas_df.drop_duplicates(inplace=True)
socios_empresas_rotuladas_df["tempo_de_sociedade"].describe()

count    1.683068e+06
mean     6.681193e+00
std      4.503041e+00
min      0.000000e+00
25%      5.000000e+00
50%      6.000000e+00
75%      7.000000e+00
max      8.600000e+01
Name: tempo_de_sociedade, dtype: float64

Mascarar CPF do sócio para facilitar join com outras bases:

In [143]:
socios_empresas_rotuladas_df['cnpj_cpf_socio_mascarado'] = np.where(np.logical_or(socios_empresas_rotuladas_df.cnpj_cpf_socio.str.len()==14,socios_empresas_rotuladas_df.cnpj_cpf_socio.str.startswith('*')), socios_empresas_rotuladas_df.cnpj_cpf_socio, '***' + socios_empresas_rotuladas_df.cnpj_cpf_socio.str[3:9] + '**')

In [144]:
socios_empresas_rotuladas_df

,cnpj,cnpj_cpf_socio,nome_socio,tempo_de_sociedade,cnpj_cpf_socio_mascarado
0,00000000000191,***923641**,MARCIO HAMILTON FERREIRA,11,***923641**
1,00000000000191,***491386**,NILSON MARTINIANO MOREIRA,11,***491386**
2,00000000000191,***718468**,WALTER MALIENI JUNIOR,11,***718468**
3,00000000000191,***415907**,CARLOS ALBERTO ARAUJO NETTO,9,***415907**
4,00000000000191,***022878**,ANTONIO MAURICIO MAURANO,9,***022878**
...,...,...,...,...,...
1687477,97541975000169,26090628859,EDIVALDO ALVES DE ARAUJO,10,***906288**
1687478,97542225000101,01095040707,CARLOS HENRIQUE ROCHA DE SOUZA,10,***950407**
1687479,97543869000114,72399317491,MARIO JORGE LIMA DA SILVA,10,***993174**
1687480,97548003000104,03028898330,CARLOS ADRIANO ROSA DA SILVA,10,***288983**


In [145]:
print(socios_empresas_rotuladas_df['cnpj_cpf_socio'].str.len().value_counts())
print(socios_empresas_rotuladas_df['cnpj_cpf_socio_mascarado'].str.len().value_counts())

11    1664526
14      18542
Name: cnpj_cpf_socio, dtype: int64
11    1664526
14      18542
Name: cnpj_cpf_socio_mascarado, dtype: int64


In [146]:
# Salvando resultado:
socios_empresas_rotuladas_df.to_csv(r'C:\DATASETS\Empresas Rotuladas\socios_empresas_rotuladas.csv', index = False)